In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gensim==3.8.1
!pip install japanize-matplotlib
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 63.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.1-cp39-cp39-linux_x86_64.whl size=24954713 sha256=bf83a19359d5633bb95d82234a3da17ac4fb3ca9a6ebcf58fd26648ffc6de4a1
  Stored in directory: /root/.cache/pip/wheels/fe/77/24/b3ffc86119703277a8aeb0e6914e97e216b1bd45287e41d6ba
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 45.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256

In [3]:
!cp -rp "/content/drive/My Drive/Colab Notebooks/cc.ja.300.vec.gz" "."

In [4]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('cc.ja.300.vec.gz')

In [5]:
# https://qiita.com/MonaCat/items/c26fa660a711f010c39a
import re
import unicodedata
import string
from janome.tokenizer import Tokenizer

j_t = Tokenizer()


def counter(word) :
    word = str(word)
    hiragana = [(i >= "ぁ") and (i <= "ゟ") for i in word].count(True)
    katakana = [(i >= "ァ") and (i <= "ヿ") for i in word].count(True)
    kanji = [(i >= "一") and (i <= "鿼") for i in word].count(True)
    return hiragana, katakana, kanji


def tokenizer_janome(text):
    toklist = []
    for tok in j_t.tokenize(text, wakati=False) :
        hinshi = tok.part_of_speech.split(',')[0]
        if hinshi == "動詞" :
            katsuyou = tok.infl_form[:2]
        elif hinshi == "形容詞" :
            katsuyou = tok.infl_form[:2]
        elif hinshi == "名詞" :
            katsuyou = tok.part_of_speech
        else :
            katsuyou = ""
        toklist.append((tok.surface, hinshi, katsuyou))
    return toklist


def format_text(text):
    text = unicodedata.normalize("NFKC", text)
    table = str.maketrans("", "", string.punctuation  + "「」、。・")
    text = text.translate(table)

    return text


def preprocessing(text):
    """
    前処理をする関数
    """
    text.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))  # 全角→半角

    text = text.lower()  # 大文字→小文字

    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub(' ', '', text)
    text = re.sub('　', '', text)

    text = re.sub(r'[0-9 ０-９]', '', text)  # 数字の除去
    text = re.sub(r'[!-/:-@[-`{-~]', '', text)  # 半角記号の除去
    text = re.sub(r'/[！-／：-＠［-｀｛-～、-〜”’・]', '', text)  # 全角記号の除去
    text = format_text(text)  # 全角記号の除去

    return  text


def tokenizer_with_preprocessing(text):
    text = preprocessing(text)
    tokens = tokenizer_janome(text)
    return tokens


def ishiragana(text) :
    for i in text :
        if ((ord("ぁ") > ord(i)) or (ord("ゟ") < ord(i))) :
            return False
    return True

import requests


def get_API(url) :
    try :
        get = requests.get(url)
    except :        # プロキシエラー等のエラーが発生したら
        print("5xx Error")
        return ""

    if (get.status_code == 200) :
        try :
            get_dict = get.json()
        except :        # JSON形式ではなかったら（メンテナンス等）
            print("Not JSON Error", get.status_code)
            return ""

        if "error" in get_dict :     # dictのキーにerrorがあったら
            print("Invalid path Error", get.status_code)
            return ""
        
        if "message" in get_dict :     # dictのキーにerrorがあったら
            print("404 on json API", get.status_code)
            
        else :      # 正常に取得できたら
            print("OK", get.status_code)
            return get_dict

    else :      # エラーステータスコードを受け取ったら（HEROKU error等）
        print("not 2xx", get.status_code)
        return ""

In [6]:
text = "吹降りを塗る、晦と落雁。閲した笛に籠り、浮き上がる様を待ちますか？薄暮に投げた、卑しい水煙が。うたた寝を繰り返さない、繋がずの大蝦蟇。寂寞の重なり合いを避けて、錆びる釣瓶落とし。刃先が寝る業晒し、絶え入りの意図でした、蘖を食べますが、身震いが起きました。因果を炊いて、辰星のクワイア。矩形波の初姿、這い上がる様が有らずとも。路地裏は解け、二十日の赤べこが。危篤の豆腐を交わさずに、懐かしむ小火騒ぎ。暮色を惜しむ事と感じます、曰く、「門前雀羅を張る恐れ」と。蘖を食べました、絶え入りの意図ですか？刃先が寝る業晒し、全て達磨の所為です。やおらに立ち竦んで、あなたは今でも浪々の身なのに、狐は荒療治で、催しは今でも猥雑の身です。蘖を食べました、絶え入りの意図ですか？刃先が寝る業晒し、全て▇▇の所為です。"
# text = "携帯ゲームの裏、フタを開けてみて、空っぽだったはずなのに、淡い光が漏れていたので、いたずらに覗いたら、デンチが腐っていた。掌から滑り落ち、叩きつけられて、やむ終えず覗いたら、画面が割れていました。たわむれに書いた傘の中、ひとりでに骨が折れ。心地よい音　頭蓋の中、湿って砕けました。湧き出た光る水を、飲んでみたくなり、空っぽだったはずなのに、器から溢れてしまいそうで、ひとくち含んでみたら、甘すぎて吐き出したよ。漏れ出た黒い液が、怖くてたまらないのに、指先が触れてしまい、血液と混ざりました。心地よい音　頭蓋の中、ひとりでに骨が折れ、たわむれに書いた傘の中、全てあなたの所為です。心地よい音　頭蓋の中、ひとりでに骨が折れ、たわむれに書いた傘の中、全て■■の所為です。沢山の目が光り、見つめていたのか。"
replaceble_hinshis = ["名詞", "動詞", "形容詞"]

In [7]:
import json

top = 10
BASE_DIR = "https://subekashi.izmn.net/api/song/"
songs = get_API(f"{BASE_DIR}?format=json")

for song in songs :
    if (song["ruigo"] == "") and song["lyrics"] and song["isjapanese"] and not(song["isjoke"]) :
        text = song["lyrics"]
        tok = tokenizer_with_preprocessing(text)
        simD = {}
        for word, hinshi, katsuyou in tok :
            if (word in model) and (hinshi in replaceble_hinshis) and sum(counter(word)) :
                simword = word
                if hinshi == "動詞" :
                    if len(word) == 1 :
                        continue
                    if word[-1] == "ん" :
                        word += "で"
                    elif word[-1] == "っ" :
                        word += "て"
                    elif word[-1] == "い" :
                        word += "て"
                elif hinshi == "形容詞" :
                    if word[-1] == "っ" :
                        word += "て"

                if ((hinshi + katsuyou) in simD.keys()) :
                    simD[hinshi + katsuyou].append(word)
                else :
                    simD[hinshi + katsuyou] = [word]

                oks = 0
                sims = model.most_similar(positive = simword, topn = top * 2)
                for sim, _ in sims :
                    _, hinshi, katsuyou = tokenizer_janome(sim)[0]
                    if (hinshi in replaceble_hinshis) and sum(counter(sim)) :
                        if hinshi == "動詞" :
                            if len(sim) == 1 :
                                continue
                            if sim[-1] == "ん" :
                                sim += "で"
                            elif sim[-1] == "っ" :
                                sim += "て"
                            elif sim[-1] == "い" :
                                sim += "て"
                        elif hinshi == "形容詞" :
                            if sim[-1] == "っ" :
                                sim += "て"
                        if ((hinshi + katsuyou) in simD.keys()) :
                            simD[hinshi + katsuyou].append(sim)
                        else :
                            simD[hinshi + katsuyou] = [sim]
                        oks += 1
                    if oks >= top :
                        break
        
        song["ruigo"] = str(simD)
        res = requests.put(f"{BASE_DIR}/{song['id']}/?format=json", data = song)
        print(song["id"], song["title"], res.status_code)
print("all done")

OK 200
386 不死 200
387 Λ [short] 200
388 珊瑚 200
389 30 200
390 明星 [short] 200
391 反復した者へと紡ぐ歌 200
392 支配 200
393 途切れなく 200
394 終末哉 200
395 往生際 200
396 イータ 200
397 アラルガンド 200
398 A⇌あ (short) 200
399 目の裏泳ぐ月光に夢見し蝶は頑なに沈む 200
400 まるで砂糖のような雨粒が 200
401 想起 200
402 叶 200
403 揺 200
404 月明かりと潜在意識 200
405 生涯、貴方を赦しません。 200
all done


In [ ]:
# import random
# lyrics = ""
# simD = simD.copy()
# tok = tokenizer_janome(text)
# for word, hinshi, katsuyou in tok :
#     if hinshi in replaceble_hinshis :
#         if (hinshi + katsuyou) in simD.keys() :
#             # fitL = [sim for sim in simD[hinshi + katsuyou] if counter(word) == counter(sim)]
#             fitL = [word]
#             for sim in simD[hinshi + katsuyou] :
#                 if (counter(word) == counter(sim)) :
#                     fitL.append(sim)
#             lyrics += random.choice(fitL)
#         else :
#             lyrics += word
#         # simD[hinshi + katsuyou].remove(sim)
#     else :
#         lyrics += word
# print(lyrics)